In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import sqlite3
import os

In [2]:
# headers = ({'User-Agent':
#             'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
#             'Accept-Language': 'en-US, en;q=0.5'})
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", 
           "Accept-Encoding":"gzip, deflate", 
           "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", 
           "DNT":"1","Connection":"close", 
           "Upgrade-Insecure-Requests":"1"}
main_page_urls = ["https://www.amazon.com/Beyond-Meat-Burger-PARENT-Frozen/dp/B074K5MDLW/"]

In [3]:
def scrape_reviews(main_page_url, headers):
    main_page = requests.get(main_page_url, headers=headers)
    main_page_soup = BeautifulSoup(main_page.content, 'html.parser')
    
    data = []
    
    product_name = main_page_soup.find('span', {'id': 'productTitle'}).get_text().strip()
    brand_name = main_page_soup.find('a', {'id': 'bylineInfo'}).get_text().strip()[7:]
    price = main_page_soup.find('span', {'id': 'priceblock_ourprice'}).get_text().strip()[1:]
    asin = main_page_soup.find('ul', {'class': 'detail-bullet-list'}).findAll('li')[4].findAll('span')[2].get_text()
    overall_rating = main_page_soup.find('span', {'data-hook': 'rating-out-of-text'}).get_text()[:3]
    
    review_url_base = main_page_url[:-1].replace("dp", "product-reviews") + "?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber="
    review_urls = [review_url_base + str(i) for i in range(1, 10)]
    
    for review_url in review_urls:
        review_page = requests.get(review_url, headers=headers)
        review_page_soup = BeautifulSoup(review_page.content, 'html.parser')
        username = review_page_soup.findAll('span', {'class': 'a-profile-name'})
        date = review_page_soup.findAll('span', {'data-hook': 'review-date'})
        rating = review_page_soup.find_all('i', {'data-hook': 'review-star-rating'})
        review = review_page_soup.find_all('span', {'data-hook': 'review-body'})
        
        for i in range(2, 8):
            try:
                reviewer_username = username[i].get_text()
                review_date = date[i].get_text()[33:]
                review_rating = rating[i].span.get_text()[:3]
                review_text = review[i].get_text()[:-9].strip()

                data.append([brand_name, product_name, asin, price, overall_rating, reviewer_username, review_date, review_rating, review_text])
                
            except:
                continue
    return data

In [4]:
def write_to_db(data):
    conn = sqlite3.connect("reviewdb.db")
    c = conn.cursor()
    c.execute("""CREATE TABLE IF NOT EXISTS scraped_reviews('brand_name', 'product_name', 'asin', 'price', 'overall_rating', 'reviewer_username', 'review_date', 'review_rating', 'review_text')""")
    
    for entry in data:
        c.execute("INSERT INTO scraped_reviews ('brand_name', 'product_name', 'asin', 'price', 'overall_rating', 'reviewer_username', 'review_date', 'review_rating', 'review_text') VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)", 
                  (entry[0], entry[1], entry[2], entry[3], entry[4], entry[5], entry[6], entry[7], entry[8]))
        
    conn.commit()
    conn.close()

In [5]:
for main_page_url in main_page_urls:
    data = scrape_reviews(main_page_url, headers)
    write_to_db(data)

In [6]:
conn = sqlite3.connect("reviewdb.db")
c = conn.cursor()
c.execute("SELECT * FROM scraped_reviews")
output = c.fetchall()
conn.close()
print(output)

[('Beyond Meat', 'Beyond Meat Beyond Burger Plant-Based Patties 2 pk, 8 oz', 'B074K5MDLW', '5.99', '4.7', 'Leon Forte', 'June 19, 2021', '2.0', 'I’ve tried this twice because I really want to move to a plant based diet but yuk, the smell is so bad, I just can’t eat it. Texture is decent, taste is ok (not like meat tho) but I just can’t stomach the'), ('Beyond Meat', 'Beyond Meat Beyond Burger Plant-Based Patties 2 pk, 8 oz', 'B074K5MDLW', '5.99', '4.7', 'Roscoe', 'June 15, 2021', '5.0', 'I really had to try these , I couldn’t believe how they tasted like a charred burger but much lighter! I cooked for only 4 mins little salt and pepper on a sesame seed bun with lettuce tomatoes and'), ('Beyond Meat', 'Beyond Meat Beyond Burger Plant-Based Patties 2 pk, 8 oz', 'B074K5MDLW', '5.99', '4.7', 'Disneymom2', 'June 15, 2021', '5.0', "I'm a vegan and I have been for years. I am happy that I finally found them at Amazon. The price is right and they taste reall"), ('Beyond Meat', 'Beyond Meat Bey